In [1]:
from pathlib import Path

import src.gdmtools as gdmtools

from cobaya.run import run
from cobaya.log import LoggedError

from classy import Class

In [2]:
PROJECT_NAME = "gdm_alpha_5w_2c_fixed_ends"


PROJECT_DIR = (Path.cwd() / PROJECT_NAME )  # Path("/opt/project/") / PROJECT_NAME / ""

CHAIN_DIR = PROJECT_DIR / "chains/"
(CHAIN_DIR / "").mkdir(exist_ok=True, parents=True)

#COBAYA_PACKAGES_PATH = Path("/software/cobaya_packages")
CLASS_PATH = "/home/mmeiers/Projects/gdm_cosmology/code/my-class" #COBAYA_PACKAGES_PATH / "code/class_public-designer/"



In [3]:
gdm_model = gdmtools.yaml.load(PROJECT_DIR / f"{PROJECT_NAME}+model.yaml")

In [4]:

lcdm_params = {
    "logA": {
        "prior": {"min": 1.61, "max": 3.91},
        "ref": {"dist": "norm", "loc": 3.05, "scale": 0.001},
        "proposal": 0.001,
        "latex": "\\log(10^{10} A_\\mathrm{s})",
        "drop": True,
    },
    "A_s": {"value": "lambda logA: 1e-10*np.exp(logA)", "latex": "A_\\mathrm{s}"},
    "n_s": {
        "prior": {"min": 0.8, "max": 1.2},
        "ref": {"dist": "norm", "loc": 0.965, "scale": 0.004},
        "proposal": 0.002,
        "latex": "n_\\mathrm{s}",
    },
    "theta_s_1e2": {
        "prior": {"min": 0.5, "max": 10},
        "ref": {"dist": "norm", "loc": 1.0416, "scale": 0.0004},
        "proposal": 0.0002,
        "latex": "100\\theta_\\mathrm{s}",
        "drop": True,
    },
    "100*theta_s": {"value": "lambda theta_s_1e2: theta_s_1e2", "derived": False},
    "H0": {"min": 60, "max": 80, "latex": "H_0"},
    "sigma8": {"latex": "\sigma_8"},
    "omega_b": {
        "prior": {"min": 0.005, "max": 0.1},
        "ref": {"dist": "norm", "loc": 0.0224, "scale": 0.0001},
        "proposal": 0.0001,
        "latex": "\\Omega_\\mathrm{b} h^2",
    },
    "omega_cdm": {
        "prior": {"min": 0.001, "max": 0.99},
        "ref": {"dist": "norm", "loc": 0.12, "scale": 0.001},
        "proposal": 0.0005,
        "latex": "\\Omega_\\mathrm{c} h^2",
    },
    "z_reio": {
        "prior": {"min": 6, "max": 9},
        "ref": {"dist": "norm", "loc": 7.5, "scale": 0.75},
        "proposal": 0.4,
    },
    "tau_reio": {"latex": "\\tau_\\mathrm{reio}"},
    "Omega_gdm_max": None,
    "z_gdm_max": None
}


In [5]:
def gdm_likeihood(_self):
    bg =_self.provider.get_CLASS_background()
    Omega_gdm = bg['(.)rho_gdm']/bg['H [1/Mpc]']**2
    Omega_gdm_max_idx = np.argmax(Omega_gdm)
    return (0, {'Omega_gdm_max': Omega_gdm[Omega_gdm_max_idx], 'z_gdm_max': bg['z'][Omega_gdm_max_idx]})

In [6]:
cobaya_info = dict(
    theory={
        "classy": {
            "extra_args": {
                **gdm_model.fixed_settings,
                "non_linear": "hmcode",
                "l_max_scalars": 5000,
            },
            #"path": str(CLASS_PATH),
            #'provide': {'get_CLASS_background': None},
        }
    },
    params={**lcdm_params, **gdm_model.params},
    likelihood={
        "planck_2018_lowl.TT": None,
        "planck_2018_lowl.EE": None,
        "planck_2018_highl_plik.TTTEEE_lite": None,
        "planck_2018_lensing.clik": None,
        'gdm': {'external': gdm_likeihood,
                'output_params': ['Omega_gdm_max','z_gdm_max'],
                'requires': {'CLASS_background': None}},
    },
    sampler=dict(
        mcmc={
            "drag": True,
            "oversample_power": 0.4,
            "proposal_scale": 1.9,
            "covmat": "auto",
            "Rminus1_stop": 0.01,
            "Rminus1_cl_stop": 0.025,
        }
    ),
    output=str(CHAIN_DIR),
    resume= True,
    #packages_path=str(COBAYA_PACKAGES_PATH),
)

In [7]:
upd_info, mcmc = run(cobaya_info)

[output] Output to be read-from/written-into folder '/home/mmeiers/Projects/gdm_cosmology/gdm-workspace/gdm_alpha_5w_2c_fixed_ends', with prefix 'chains'
[output] Found existing info files with the requested output prefix: '/home/mmeiers/Projects/gdm_cosmology/gdm-workspace/gdm_alpha_5w_2c_fixed_ends/chains'
[output] Let's try to resume/load.
[output] Found an old sample. Resuming.
[classy] `classy` module loaded successfully from /home/mmeiers/anaconda3/envs/py39/lib/python3.9/site-packages/classy-3.2.0-py3.9-linux-x86_64.egg
/home/mmeiers/Projects/gdm_cosmology/code/planck/code/plc_3.0/plc-3.1 3.1
[planck_2018_lowl.tt] `clik` module loaded successfully from /home/mmeiers/Projects/gdm_cosmology/code/planck/code/plc_3.0/plc-3.1/lib/python/site-packages/clik
----
clik version plc_3.1
  gibbs_gauss b13c8fda-1837-41b5-ae2d-78d6b723fcf1
Checking likelihood '/home/mmeiers/Projects/gdm_cosmology/data/planck_2018/baseline/plc_3.0/low_l/commander/commander_dx12_v3_2_29.clik' on test data. got 

In [ ]:
from classy import Class

bad_in = gdm_model.fixed_settings|{'output':"mPk",'A_s': 2.1130577961081382e-09, 'n_s': 0.9688121598431281, '100*theta_s': 1.042027519819202, 'omega_b': 0.022350917183541954, 'omega_cdm': 0.11983305955937719, 'z_reio': 8.056572538611144, 'gdm_alpha': 0.05944613357551692, 'gdm_w_vals': '-1.0,0.29195379412124445,-0.087584465639726,0.294193827530018,-0.09145501866764416,0.32421587825205,1.0', 'gdm_c_eff2': 0.01811824380355831, 'gdm_c_vis2': 0.03961205463083206, 'gdm_z_alpha': 3000.0}
cosmo = Class()
cosmo.set(bad_in)
cosmo.compute()
bg = cosmo.get_background()
max(bg['(.)rho_gdm']/bg['H [1/Mpc]']**2)




0.06586054424131565

In [ ]:
bg

{'z': array([1.00000000e+14, 9.89308584e+13, 9.78731474e+13, ...,
        2.17307062e-02, 1.08069579e-02, 0.00000000e+00]),
 'proper time [Gyr]': array([7.55951851e-26, 7.72379221e-26, 7.89163568e-26, ...,
        1.27633742e+01, 1.28995757e+01, 1.30363223e+01]),
 'conf. time [Mpc]': array([4.63478500e-09, 4.68571670e-09, 4.73664841e-09, ...,
        1.37100208e+04, 1.37524592e+04, 1.37946119e+04]),
 'H [1/Mpc]': array([2.15725629e+22, 2.11137463e+22, 2.06646880e+22, ...,
        2.57924623e-04, 2.56884101e-04, 2.55872569e-04]),
 'comov. dist.': array([13794.61186938, 13794.61186938, 13794.61186938, ...,
           84.59107424,    42.15268161,     0.        ]),
 'ang.diam.dist.': array([1.37946119e-10, 1.39436897e-10, 1.40943785e-10, ...,
        8.27919468e+01, 4.17020097e+01, 0.00000000e+00]),
 'lum. dist.': array([1.37946119e+18, 1.36471279e+18, 1.35012208e+18, ...,
        8.64292980e+01, 4.26082239e+01, 0.00000000e+00]),
 'comov.snd.hrz.': array([2.67631746e-09, 2.70524032e-09, 2.

In [ ]:
model = get_model(cobaya_info)

NameError: name 'get_model' is not defined

In [ ]:
from classy import Class

test_in = {'gdm_alpha': 0.05944613357551692,
           'gdm_c_eff2' : 0.01811824380355831,
           'gdm_c_vis2' : 0.03961205463083206,
           'gdm_log10a_vals' : '-14,-4.5,-4.0,-3.5,-3.0,-2.5,0',
           'gdm_w_vals' : '-1.0,0.29195379412124445,-0.087584465639726,0.294193827530018,-0.09145501866764416,0.32421587825205,1.0',
           'gdm_z_alpha' : 3000,
           'gdm_interpolation_order' :1,
           'output': 'mPk',
           'A_s': 2.1130577961081382e-09,
           'n_s': 0.9688121598431281,
           '100*theta_s': 1.042027519819202,
           'omega_b': 0.022350917183541954,
           'omega_cdm': 0.11983305955937719,
           'z_reio': 8.056572538611144,
           }

cosmo = Class()
cosmo.set(test_in)
#cosmo.set(bad_in)
cosmo.compute()
cosmo.struct_cleanup()
cosmo.empty()

In [ ]:
gdm_model.fixed_settings

{'gdm_log10a_vals': '-14,-4.5,-4.0,-3.5,-3.0,-2.5,0',
 'gdm_interpolation_order': 1}

In [ ]:
test = np.array(1)
np.argmax(np.arange(5), out = test)

array(4)

In [ ]:
test

array(4)